<a href="https://colab.research.google.com/github/HarryChintam/StreamLitPoc/blob/CodePoc/StreamLit_Poc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
!pip install streamlit pyngrok

# Step 2: Configure ngrok with your authtoken
!ngrok config add-authtoken 2oRSv8BtAHwfqAnArqee5dOi4NL_7dViZ8N5Y8imAu6YTzofr

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [51]:
!streamlit run app.py &>/dev/null&

In [56]:

# Step 3: Write the Streamlit application code to a file
app_code = """
import streamlit as st
import pandas as pd

# Define the data as a dictionary
data_dict = {
    'model_forecasted_dates': ['January', 'February', 'March', 'April', 'May'],
    'forecast_amount': [100, 150, 200, 250, 300],
    'actual_user_input_amount': [50, 75, 100, 125, 150],
    'user_adjustment': [0,0,0,0,0]
}

# Convert the dictionary to a Pandas DataFrame
data = pd.DataFrame(data_dict)

st.title("Update Actual Data for the Marvel Finance - Movie Forecasted data for each model_forecasted_dates")

# Display the current data
st.write("Current Data", data)

# Create input fields for user to update data
model_forecasted_dates = st.selectbox("Select model_forecasted_dates", options=data['model_forecasted_dates'].unique())

# Get the current forecast value for the selected model_forecasted_dates
current_forecast_amount = data.loc[data['model_forecasted_dates'] == model_forecasted_dates, 'forecast_amount'].values[0]
current_actual_user_input_amount = data.loc[data['model_forecasted_dates'] == model_forecasted_dates, 'actual_user_input_amount'].values[0]

# Pre-fill the number_input with the current forecast value and actual value
forecast_amount = st.number_input("Forecast Value", value=current_forecast_amount)
actual_user_input_amount = st.number_input("Actual Value", value=current_actual_user_input_amount)

# forecast_amount = st.selectbox("Select forecast_amount", options=data['forecast_amount'].unique())
# forecast_amount = st.number_input("Forecast Value", value=0)
# actual_user_input_amount = st.number_input("actual Field", value=0)

# Dynamic calculation (example)
user_adjustment = forecast_amount + actual_user_input_amount  # Example calculation
st.write(f"Dynamic Field: {user_adjustment}")

# Update the DataFrame
if st.button("Submit"):
    # Update the DataFrame
    data.loc[data['model_forecasted_dates'] == model_forecasted_dates, 'forecast_amount'] = forecast_amount
    data.loc[data['model_forecasted_dates'] == model_forecasted_dates, 'actual_user_input_amount'] = actual_user_input_amount
    data.loc[data['model_forecasted_dates'] == model_forecasted_dates, 'user_adjustment'] = user_adjustment

    # Display the updated DataFrame
    st.write("Updated Data", data)
    st.success("Data updated successfully!")
"""

with open("app.py", "w") as file:
    file.write(app_code)


In [62]:
# Step 3: Write the Streamlit application code to a file
app_code = """
import streamlit as st
import pandas as pd
from datetime import datetime

# Initialize session state for data
if 'data' not in st.session_state:
    data_dict = {
        'movie_title': ['Avengers', 'Avengers', 'Avengers', 'Avengers', 'Millionaire'],
        'movie_phase': ['Prep', 'Prep', 'Prep', 'Prep', 'Release'],
        'model_forecasted_dates': ['Nov-18', 'Nov-18', 'Nov-18', 'Nov-18', 'Nov-18'],
        'model_forecasted_date': ['Nov-21', 'Nov-22', 'Nov-23', 'Nov-24', 'Nov-25'],
        'forecast_amount': [100, 105, 110, 114, 134],
        'actual_user_input_amount': [0, 0, 0, 0, 0],
        'user_adjustment': [0, 0, 0, 0, 0],
    }
    st.session_state.data = pd.DataFrame(data_dict)

data = st.session_state.data

st.title("Update Actual amount for each phase of the security title against Model Forecast data")

# Display the current data
st.write("Current Data", data)

# Create filter for movie_title
selected_movie_title = st.selectbox("Select movie_title", options=data['movie_title'].unique())

# Filter data based on selected movie_title
filtered_data = data[data['movie_title'] == selected_movie_title]

# Create filter for movie_phase
selected_movie_phase = st.selectbox("Select movie_phase", options=filtered_data['movie_phase'].unique())

# Filter data based on selected movie_phase
filtered_data = filtered_data[filtered_data['movie_phase'] == selected_movie_phase]

# Create input fields for user to update data
selected_model_forecasted_dates = st.multiselect("Select model_forecasted_date", options=filtered_data['model_forecasted_date'].unique())

if selected_model_forecasted_dates:
    with st.form(key='update_form'):
        updated_data = {}
        for model_forecasted_date in selected_model_forecasted_dates:
            st.write(f"Updating data for {model_forecasted_date}")
            current_forecast_amount = filtered_data.loc[filtered_data['model_forecasted_date'] == model_forecasted_date, 'forecast_amount'].values[0]
            current_actual_user_input_amount = filtered_data.loc[filtered_data['model_forecasted_date'] == model_forecasted_date, 'actual_user_input_amount'].values[0]

            # Display forecast_amount but make it non-editable
            st.write(f"Forecast Value for {model_forecasted_date}: {current_forecast_amount}")
            actual_user_input_amount = st.number_input(f"Actual Value for {model_forecasted_date}", value=current_actual_user_input_amount, key=f"actual_{model_forecasted_date}")

            # Dynamic calculation (example)
            user_adjustment = actual_user_input_amount - current_forecast_amount  # Example calculation
            st.write(f"Dynamic Field for {model_forecasted_date}: {user_adjustment}")

            # Store the updated values in a dictionary
            updated_data[model_forecasted_date] = {
                'forecast_amount': current_forecast_amount,
                'actual_user_input_amount': actual_user_input_amount,
                'user_adjustment': user_adjustment
            }

        # Update the DataFrame when the form is submitted
        if st.form_submit_button("Submit"):
            for model_forecasted_date, values in updated_data.items():
                data.loc[(data['movie_title'] == selected_movie_title) &
                         (data['movie_phase'] == selected_movie_phase) &
                         (data['model_forecasted_date'] == model_forecasted_date), 'actual_user_input_amount'] = values['actual_user_input_amount']
                data.loc[(data['movie_title'] == selected_movie_title) &
                         (data['movie_phase'] == selected_movie_phase) &
                         (data['model_forecasted_date'] == model_forecasted_date), 'user_adjustment'] = values['user_adjustment']

            st.session_state.data = data
            st.write("Updated Data", st.session_state.data)
            st.success("Data updated successfully!")
            st.session_state.show_refresh = True

# Display the refresh button if data was updated
if st.session_state.get('show_refresh', False):
    if st.button("Refresh Data"):
        st.write("Current Data", st.session_state.data)
        st.session_state.show_refresh = False
"""

with open("app.py", "w") as file:
    file.write(app_code)

In [59]:
# Step 4: Use ngrok to expose the Streamlit app
from pyngrok import ngrok

# Start ngrok tunnel
public_url = ngrok.connect(addr='8501')
print(f"Streamlit app is live at: {public_url}")

# Step 5: Run the Streamlit app
!streamlit run app.py &>/dev/null&

Streamlit app is live at: NgrokTunnel: "https://4fc8-34-122-196-255.ngrok-free.app" -> "http://localhost:8501"
